In [ ]:
import pandas as pd


user_details = pd.read_excel("/content/drive/MyDrive/Upliance.ai/Assignment.xlsx", sheet_name=0)
cooking_sessions = pd.read_excel("/content/drive/MyDrive/Upliance.ai/Assignment.xlsx", sheet_name=1)
order_details = pd.read_excel("/content/drive/MyDrive/Upliance.ai/Assignment.xlsx", sheet_name=2)

In [ ]:
user_details.isnull().sum()

,0
User ID,0
User Name,0
Age,0
Location,0
Registration Date,0
Phone,0
Email,0
Favorite Meal,0
Total Orders,0


In [ ]:
cooking_sessions.isnull().sum()

,0
Session ID,0
User ID,0
Dish Name,0
Meal Type,0
Session Start,0
Session End,0
Duration (mins),0
Session Rating,0


In [ ]:
order_details.isnull().sum()

,0
Order ID,0
User ID,0
Order Date,0
Meal Type,0
Dish Name,0
Order Status,0
Amount (USD),0
Time of Day,0
Rating,0
Session ID,0


In [ ]:
order_details['Rating'].fillna(order_details['Rating'].mean(), inplace=True)

<ipython-input-10-1bd53fa3365b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  order_details['Rating'].fillna(order_details['Rating'].mean(), inplace=True)


In [ ]:
sessions_users = pd.merge(cooking_sessions, user_details, on='User ID', how='inner')

In [ ]:
df = pd.merge(sessions_users, order_details, on=['Session ID', 'User ID', 'Dish Name', 'Meal Type'], how='inner')

In [ ]:
df.head()

,Session ID,User ID,Dish Name,Meal Type,Session Start,Session End,Duration (mins),Session Rating,User Name,Age,...,Phone,Email,Favorite Meal,Total Orders,Order ID,Order Date,Order Status,Amount (USD),Time of Day,Rating
0,S001,U001,Spaghetti,Dinner,2024-12-01 19:00:00,2024-12-01 19:30:00,30,4.5,Alice Johnson,28,...,123-456-7890,alice@email.com,Dinner,12,1001,2024-12-01,Completed,15.0,Night,5.000000
1,S002,U002,Caesar Salad,Lunch,2024-12-01 12:00:00,2024-12-01 12:20:00,20,4.0,Bob Smith,35,...,987-654-3210,bob@email.com,Lunch,8,1002,2024-12-01,Completed,10.0,Day,4.000000
2,S003,U003,Grilled Chicken,Dinner,2024-12-02 19:30:00,2024-12-02 20:10:00,40,4.8,Charlie Lee,42,...,555-123-4567,charlie@email.com,Breakfast,15,1003,2024-12-02,Canceled,12.5,Night,4.285714
3,S004,U001,Pancakes,Breakfast,2024-12-02 07:30:00,2024-12-02 08:00:00,30,4.2,Alice Johnson,28,...,123-456-7890,alice@email.com,Dinner,12,1004,2024-12-02,Completed,8.0,Morning,4.000000
4,S005,U004,Caesar Salad,Lunch,2024-12-03 13:00:00,2024-12-03 13:15:00,15,4.7,David Brown,27,...,444-333-2222,david@email.com,Dinner,10,1005,2024-12-03,Completed,9.0,Day,4.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Session ID         16 non-null     object        
 1   User ID            16 non-null     object        
 2   Dish Name          16 non-null     object        
 3   Meal Type          16 non-null     object        
 4   Session Start      16 non-null     datetime64[ns]
 5   Session End        16 non-null     datetime64[ns]
 6   Duration (mins)    16 non-null     int64         
 7   Session Rating     16 non-null     float64       
 8   User Name          16 non-null     object        
 9   Age                16 non-null     int64         
 10  Location           16 non-null     object        
 11  Registration Date  16 non-null     datetime64[ns]
 12  Phone              16 non-null     object        
 13  Email              16 non-null     object        
 14  Favorite Mea

In [ ]:
df['Order ID'] = df['Order ID'].astype(str)

In [ ]:
df.to_csv('cleaned_excel.csv')

In [ ]:
import plotly.express as px

cooking_order_correlation = df.groupby('Dish Name')['Order ID'].count().reset_index()
cooking_order_correlation = cooking_order_correlation.sort_values(by='Order ID', ascending=False)

fig = px.bar(cooking_order_correlation, x='Dish Name', y='Order ID',
             title="Frequency of Dishes Ordered",
             text='Order ID', color='Order ID', color_continuous_scale='Cividis')
fig.update_layout(xaxis_title="Dish Name", yaxis_title="Order Frequency")
fig.show()


In [ ]:
revenue_by_dish = df.groupby('Dish Name')['Amount (USD)'].sum().reset_index()
revenue_by_dish = revenue_by_dish.sort_values(by='Amount (USD)', ascending=True)

fig = px.bar(revenue_by_dish, x='Amount (USD)', y='Dish Name', orientation = 'h', title="Revenue by Dishes",
             text='Amount (USD)', color='Amount (USD)', color_continuous_scale='Viridis')
fig.update_layout(xaxis_title="Dish Name", yaxis_title="Total Revenue (USD)")
fig.show()

In [ ]:
fig = px.box(df, x='Session Rating', title="Distribution of Cooking Session Ratings",
             color_discrete_sequence=['purple'])
fig.update_layout(xaxis_title="Session Rating")
fig.show()

In [ ]:
df['Order Frequency'] = df.groupby('User ID')['Order ID'].transform('count')

fig = px.scatter(df, x='Duration (mins)', y='Order Frequency',
                 title="Time Spent Cooking vs Order Frequency",
                 color='Order Frequency', size='Order Frequency',
                 labels={'Duration (mins)': 'Duration (mins)', 'Order Frequency': 'Order Frequency'},
                 color_continuous_scale='Bluered')
fig.update_layout(xaxis_title="Duration (mins)", yaxis_title="Order Frequency")
fig.show()

In [ ]:
df['Age Group'] = pd.cut(df['Age'], bins=[0, 20, 30, 40, 50, 100],
                                   labels=['0-20', '20-30', '30-40', '40-50', '50+'])

age_orders = df.groupby('Age Group')['Order ID'].count().reset_index()
age_orders.columns = ['Age Group', 'Order Count']

fig = px.bar(age_orders, x='Age Group', y='Order Count',
             title="Orders by Age Group", text='Order Count',
             color='Age Group', color_discrete_sequence=px.colors.qualitative.Dark2)
fig.update_layout(xaxis_title="Age Group", yaxis_title="Order Count")
fig.show()

<ipython-input-67-5215d1479996>:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
meal_type_counts = cooking_sessions['Meal Type'].value_counts().reset_index()

fig = px.pie(meal_type_counts, names='Meal Type', values='count', title="Cooking Sessions by Meal Type",
              color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(xaxis_title="Meal Type", yaxis_title="Number of Sessions")
fig.update_traces(textinfo='percent+label')
fig.show()

In [ ]:
fig = px.box(df, x='Meal Type', y='Duration (mins)',
             title="Cooking Session Duration by Meal Type",
             color='Meal Type', color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_layout(xaxis_title="Meal Type", yaxis_title="Duration (mins)")
fig.show()

In [ ]:
dish_analysis = df.groupby('Dish Name').agg({'Rating': 'mean',
                                                      'Amount (USD)': 'sum'}).reset_index()
dish_analysis.columns = ['Dish Name', 'Average Rating', 'Total Revenue']

fig = px.scatter(dish_analysis, x='Average Rating', y='Total Revenue',
                 size='Total Revenue', color='Average Rating',
                 title="Ratings vs. Revenue per Dish",
                 labels={'Average Rating': 'Average Dish Rating', 'Total Revenue': 'Total Revenue (USD)'},
                 color_continuous_scale='Magma')
fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(xaxis_title="Average Rating", yaxis_title="Total Revenue (USD)")
fig.show()


In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
orders_trend = df.groupby(df['Order Date'].dt.to_period('D')).size().reset_index(name='Order Count')
orders_trend['Order Date'] = orders_trend['Order Date'].astype(str)

fig = px.line(orders_trend, x='Order Date', y='Order Count',
              title="Trend of Orders Over Time")
fig.update_traces(line=dict(color='blue', width=2))
fig.update_layout(xaxis_title="Date", yaxis_title="Number of Orders")
fig.show()

In [ ]:
df['Hour Cooked'] = df['Session Start'].dt.hour

orders_by_hour = df.groupby('Hour Cooked').size().reset_index(name='Order Count')

fig = px.line(orders_by_hour, x='Hour Cooked', y='Order Count',
              title="Trend of Orders Over the Hour Cooked")
fig.update_layout(xaxis_title="Hour of Cooking Session", yaxis_title="Number of Orders")
fig.update_traces(line=dict(color='green', width=2))
fig.show()

In [ ]:
order_status_counts = df['Order Status'].value_counts().reset_index()
order_status_counts.columns = ['Order Status', 'Count']

fig = px.pie(order_status_counts, values='Count', names='Order Status', title="Order Status Breakdown",
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_traces(textinfo='percent+label')
fig.show()

In [ ]:
order_status_trend = order_details.groupby([order_details['Order Date'].dt.to_period('D'), 'Order Status']).size().reset_index(name='Order Count')
order_status_trend['Order Date'] = order_status_trend['Order Date'].astype(str)

fig = px.line(order_status_trend, x='Order Date', y='Order Count', color='Order Status',
              title="Order Status Trend Over Time",
              labels={'Order Date': 'Date', 'Order Count': 'Number of Orders'})


fig.update_traces(line=dict(width=2))
fig.update_layout(title="Order Status Trend Over Time",
                  xaxis_title="Date",
                  yaxis_title="Number of Orders",
                  legend_title="Order Status")

fig.show()